# JupyLabBook @ SIRIUS

In [1]:
import CustomFunctions as CF
import FrontendFunctions as FF
import PyNexus as PN

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
    
class Experiment:
    """
    Class Experiment is used to pass arguments concerning the current experiment only.
    """
    def __init__(self):
        pass
    
expt = Experiment() 

# Necessary for saving in pdf
expt.notebook_name = 'JupyLabBook.ipynb'

# Directory where the data will be saved
expt.working_dir = "/Users/arnaudhemmerle/Documents/Recherche/Analysis/JupyLabBook/working/"

# Directory where the nexus files are 
expt.recording_dir = "/Users/arnaudhemmerle/Documents/Recherche/Analysis/JupyLabBook/recording/"

# Directory where the logs are
expt.logs_dir = "/Users/arnaudhemmerle/Documents/Recherche/Analysis/JupyLabBook/logs/"   

######################################
FF.Check_files(expt)

Versions of modules used:
CustomFunctions: 0.17
FrontendFunctions: 0.17
PyNexus: v4.1
Check that you are using the last versions of the modules and read the manual on: 
https://github.com/ArnaudHemmerle/JupyLabBook



<IPython.core.display.Javascript object>

### SIRIUS_2020_03_11_0744: dscan basez -.2 .2 50 .1

In [7]:
FF.Choose_treatment(expt)

Selected scans:


### SIRIUS_2020_03_12_0760: run cont_regh.ipy

In [10]:
FF.Choose_treatment(expt)

interactive(children=(Dropdown(description='xLabel', options=('delta', 'zs', 'gamma', 'hu36energy', 'xs', 'ene…

### SIRIUS_2020_03_12_0760: run cont_regh.ipy

In [8]:
FF.Choose_action(expt)

interactive(children=(SelectMultiple(description='nxs_files', options=('SIRIUS_Isotherm_2019_02_17_01544.nxs',…